---
title: Some List Prooofs in Knuckledragger
---

Lists are pretty simple.

What if I attached def to the object it is defining

`mem.defn = axiom()`
AstRef.proofs = lambda self: for k,p in self.__dict__.items() if isinstance(p,Proof) yield k, pf


In [ ]:
import knuckledragger as kd
def T(S : SortRef):
    L = Datatype("List<{}>".format(S.name()))
    L.declare("nil")
    L.declare("cons", ("head", S), ("tail", L))
    L = L.create()

def induct(S : SortRef):
    L = T(S)
    return kd.define('induct', [P],  Implies(P[L.nil] & QForAll([h, l], P[l], P[L.cons(h, l)]), ForAll([l], P[l])))


def List(S :SortRef) -> DatatypeSortRef:
    """convernience function packaging everything up"""
    T = T(S)
    T.induct = induct(S)
    return T    



In [8]:
from z3 import *
from knuckledragger import *
from knuckledragger.notation import QForAll
def List(S : SortRef) -> DatatypeSortRef:
    L = Datatype("List<{}>".format(S.name()))
    L.declare("nil")
    L.declare("cons", ("head", S), ("tail", L))
    L = L.create()
    P = Const("P", ArraySort(L, BoolSort()))
    l = Const("l", L)
    h = Const("h", S)
    # hmm. But I want a polymorphic definition.
    L.induct = define('induct', [P],  Implies(P[L.nil] & QForAll([h, l], P[l], P[L.cons(h, l)]), ForAll([l], P[l])))
    def rec(T): # polymiorphioc return type
        b = Const("b", T)
        s = Array("s", S, L, T)
        rec = Function(f"rec<{T}>", T, s.sort(), ArraySort(L, T))
        define(f"rec<{T}>", [b,s], Lambda([l], If(L.is_nil(l), b, s[L.car(l), rec(b,s)[L.cdr(l)])])
    def fold(base,f):
        
    L.rec = rec #define('rec', L, [b,s],  If(L.is_nil(b), b, s[L.cons(L.car(b), L.cdr(b))]))
    return L

List(DeclareSort("T"))

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (3084730785.py, line 18)

https://microsoft.github.io/z3guide/docs/theories/Sequences/
map nth fold foldi. Looks like these may not be exposed. Bummer.
https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-sequences-and-strings


In [26]:
IntSeq = SeqSort(IntSort())
l = Const("l", IntSeq)
print(simplify(Length(l))) 
l == Empty(IntSeq)
solve(l == Unit(IntVal(3)))
prove(Contains(l,l))
# SuffixOf PrefixOf
print(simplify(IndexOf(Unit(IntVal(3)),Unit(IntVal(3)))))
t = IntVal(2)
simplify(Concat(Unit(t),Unit(t), Unit(t)))
x = Int("x")
f = Function("f", IntSort(), IntSort())
#Map(f, l) doesn't work
l[0]
#dir(l)


Length(l)
[l = Unit(3)]
proved
0


['__add__',
 '__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'arg',
 'as_ast',
 'as_string',
 'ast',
 'at',
 'children',
 'ctx',
 'ctx_ref',
 'decl',
 'eq',
 'from_string',
 'get_id',
 'hash',
 'is_string',
 'is_string_value',
 'num_args',
 'params',
 'serialize',
 'sexpr',
 'sort',
 'sort_kind',
 'translate',
 'use_pp']

In [ ]:
def induct_seq(T):
    S = Seq(T)
    def res(P):
        return Implies(And(P[Empty(S)], P()kd.QForAll([x, y], P(x), P(y) P[Unit(t) + l]), ForAll([l], P[l]))


Hmm. Actually.


In [ ]:

    def Const(name):
        x = Const(name, L)
        x.__class__ = ListRef
        return x



class ListRef(DatatypeRef):
    def car(self):
        return self.sort().accessor(0,0)(self)
    def cdr(self):
        return self.sort().accessor(0,1)(self)
    def is_nil(self):
        return self.sort().recognizer(0)(self)
    def is_cons(self):
        return self.sort().recognizer(1)(self)

datatypes = {}

def Inductive():


def overload(name, sort, f):
    r = getattr(DataTypeRef, name)
    if r == None:
        d = SortDispatch(sort)
        setattr(DataTypeRef, name, d)
        d.register(sort, f)
    if r isinstance(r, SortDispatch):
        r = r(sort)

#or constructor in constructors:
#    overload(constructor.name, constructor.sort, lambda constructor)
for recognizer in recognizers:
    overload(recognizer.name, sort, lambda x: recognizer(x))
for accessor in accessors:
    overload(accessor.name, sort, property(lambda x: accessor(x)))


# If I make a register function, I don't have to overload Datatype
def register_datatype(dt):
    datatypes[dt.name()] = dt
    for recognizer in dt.recognizers():
        overload(recognizer.name, dt, lambda x: recognizer(x))
    for accessor in dt.accessors():
        overload(accessor.name, dt, property(lambda x: accessor(x)))




In [ ]:

"""
Tiny helper. But I need more pretty fast.
Ah, but it is a bit odd to write, but cleaner to use to just attach stuff to this object.

def List(s : SortRef):
    L = Datatype(f"List_{s}")
    L.declare("nil")
    L.declare("cons", ("car", s), ("cdr", L))
    L = L.create()


    L.induct = 
    L.from_list =
    return L

"""
# https://coq.inria.fr/doc/master/stdlib/Coq.Lists.List.html
class List():
    def __init__(self, elt : SortRef, admit=False):
        L = Datatype(f"List_{elt}")
        L.declare("nil")
        L.declare("cons", ("car", elt), ("cdr", L))
        L = L.create()
        
        self.elt = elt 
        self.t = L
        
        """
        This is just bulk junk.
        I could maybe just subclass DatatypeSortRef?
        self.nil = L.nil
        self.cons = L.cons
        self.car = L.car
        self.cdr = L.cdr
        self.is_cons = L.is_cons
        self.
        """
        x,y,z,l = Consts("x y z l", L)
        a,b,c = Consts("a b c", elt)

        self.append = Function("append", L, L, L)
        self.append_def = ForAll([x, y], self.append(x, y) == If(L.is_nil(x), y,
                                                                 L.cons(L.car(x), self.append(L.cdr(x), y))))
        self.app_nil_l = lemma(ForAll([x], self.append(L.nil, x) == x), by=[self.append_def])
        self.app_nil_r = lemma(ForAll([x], self.append(x, L.nil) == x), by=[self.append_def, self.induct(lambda x : self.append(x, L.nil) == x)])
        self.append_assoc = lemma(ForAll([x,y,z], self.append(x, self.append(y,z)) == self.append(self.append(x,y), z)), admit=True)

        self.mem = Function("mem", L, elt, BoolSort())
        self.mem_def = ForAll([a, l], self.mem(l, a) == If(L.is_nil(l), False,
                                                        If(L.car(l) == a, True, self.mem(L.cdr(l), a))))
        
        self.rev = Function("rev", L, L)
        self.rev_def = ForAll([l], self.rev(l) == If(L.is_nil(l), l, 
                                                     self.append(self.rev(L.cdr(l)), L.cons(L.car(l), L.nil))))
        
        # map, filter
    def Const(self, name):
        # could I attach operator overloads? It'd be really nice.
        return Const(name, self.t)
        

    def induct(self, P):
        x = FreshConst(self.elt, prefix="x")
        l = FreshConst(self.t, prefix="l")
        return Implies(And(
                        P(self.t.nil),
                        ForAll([x,l], Implies(P(l), P(self.t.cons(x, l))))),
                       #-----------------------------------
                        ForAll([l], P(l)))
    def from_list(self,l : list[ExprRef]):
        if len(l) == 0:
            return self.sort.nil
        return self.sort.cons(l[0], self.from_list(l[1:]))
# Running over this should be sufficient, because List_Dummy has no properties
# This is a meta argument though.
List(DeclareSort("List_Dummy"))

In [ ]:
sorted_ = Function("sorted", IntList, BoolSort())



In [12]:
def List(S):
    L = Datatype(f"List[{S.name()}]")
    L.declare('nil')
    L.declare('cons', ('car', S), ('cdr', L))
    L = L.create()
    def induct(P):
        x = FreshConst('x', S)
        l = FreshConst('l', L)
        return Implies(And(P(L.nil), ForAll([x, l], Implies(P(l), P(L.cons(x, l))))),
                       ForAll([l], P(l)))
    # def recurse(base, step):
    return L, induct

IntList, intlist_induct,  = List(IntSort())




NameError: name 'List' is not defined

In [ ]:
@dataclass(frozen=True)
class Nat():
    x : IntRef
    pos : BoolRef # Proof
__Nat = Nat

# refinement types.
def Nat(x : IntRef, by=[]):
    assert isinstance(x, IntRef)
    return __Nat(x, lemma(x >= 0, by))

@dataclass(frozen=True)
class Pos(Nat): # playing with fire here.
    pos : BoolRef
__Pos = Pos
def Pos(x : IntRef, by=[]):
    assert isinstance(x, IntRef)
    return __Pos(x, lemma(x > 0, by))

# Older

So last time I blasted in a definition of peano addition to knuckledragger as an axiom. This is fine, but probably not the best.

It is totally fine for fooling around to add new axioms to expediate getting to whatever meat you want to get to. I recall it being a revelation when I saw Cody do that in Coq, that I didn't need to start from inductive datatypes and build a whole world, one can just posit new axioms.

However, it is nice to have the option to instead use more controlled or opinionated interfaces to add axioms.

When you define an inductive datatype in Coq or lean, it says that the system has defined a couple of primitives and helpers for you.

Inductive datatypes by design allow for admitting induction principles.

Since Z3 has support for inductive data types, it seems nice to piggy back on this mechanism

A classic thing to do is create a conservative extension to a theory. This adds new constants and function symbols to the signature, but no new theorems involving only old constants are provable.
See the metamath book section 4.4 <https://us.metamath.org/downloads/metamath.pdf>
<https://en.wikipedia.org/wiki/Conservative_extension>
[FOM: Conservative Extension](https://cs.nyu.edu/pipermail/fom/1998-October/002306.html)

When you intermix inductive datatypes and first class functions things get dicey for some reason. There are certain positivity restrictions to keep things sound.
Mutually recursive datatypes is also a fiddly area.
<https://en.wikipedia.org/wiki/Induction-induction>
<https://en.wikipedia.org/wiki/Induction-recursion>
<https://proofassistants.stackexchange.com/questions/926/what-are-well-founded-inductive-types>

<http://adam.chlipala.net/papers/PhoasICFP08/> HOAS is fiddly
<https://cstheory.stackexchange.com/questions/22157/w-types-vs-inductive-types>
<https://mathoverflow.net/questions/402435/why-are-w-types-called-w> W types fit into this discussion somewhere <https://ncatlab.org/nlab/show/W-type>


In [ ]:
from knuckledragger.kernel import *
from z3 import *
class Datatype(z3.Datatype):
    def create(self):
        DT = super().create()  # z3 already checks for positivity.
        PredSort = ArraySort(DT, BoolSort())
        # construct an induction principle.
        P = FreshConst(PredSort, prefix="P")
        hyps = []
        for i in range(DT.num_constructors()):
            constructor = DT.constructor(i)
            args = [
                FreshConst(constructor.domain(j), prefix="a")
                for j in range(constructor.arity())
            ]
            acc = P[constructor(*args)]
            for arg in args:
                if arg.sort() == DT:
                    acc = QForAll([arg], P[arg], acc)
                else:
                    acc = ForAll([arg], acc)
            hyps.append(acc)
        x = FreshConst(DT, prefix="x")
        conc = ForAll([x], P[x])
        induct = Lambda([P], Implies(And(hyps), conc))
        induct_ax = trust(ForAll([P], induct[P] == True))

        # recursor

        # store inside the data type?
        DT.induct = induct
        DT.induct_ax = induct_ax
        #DT.rec = rec
        return DT

def recursor(name, *cases, DT):
    assert all(case.range() == DT for case in cases)
    f = z3.RecFunction(name, DT, )  # also freshness needs to be checked


def define_rec(name, args, body, measure=None):
    sig = [arg.sort() for arg in args]
    f = z3.RecFunction(name, *sig)  # also freshness needs to be checked. Yikes.
    # TODO check termination and totality of definition
    RecAddDefinition(f, body)
    fun_def = infer([], ForAll(args, f(*args) == body))
    return f, fun_def

Wrappers for algebraic datatypes deriving induction principles, recursion principles, others? Some kind of well foundedness? What does isabelle define?

In [ ]:
List = Datatype("List")
List.declare("cons", ("car", IntSort()), ("cdr", List))
List.declare("nil")
List = List.create()
List.induct_ax

List2 = Datatype("List")
List2.declare("nil")
List2 = List2.create()
List2.induct_ax

List2 == List # ruh roh

List.induct_ax
List2.induct_ax
prove(List2.nil == List.nil) # yikes
List.num_constructors()
List2.num_constructors()
# ok so we overwrite the old list.

# This possibly invlaidates any oldtheorem

I guess one kind of insane option is to allow delayed validation.

We might need to rexport everything from z3 with some stuff wrapped. That's a somewhat nuclear options.
Or cache when it comes into trust?
An even more nuclear option is make my own adt